In [1]:
import json
import pyspark.sql.functions as f

StatementMeta(, , , Cancelled, )

In [1]:
#PARAMETERS
folderpath = ""
filename = ""
mapping = ""
sinkdbTable = ""

StatementMeta(defaultSpark33, 4, 1, Finished, Available)

In [ ]:
#Storage Config
storageLinkedService = 'LS_DataLake'
storageAccount_ls = mssparkutils.credentials.getPropertiesAll(storageLinkedService)
storage_account = json.loads(storageAccount_ls)['Endpoint'].split('.')[0].replace('https://','')
container = 'raw'

In [ ]:
# ## TEST PARAMETERS
# container = "raw"
# folderpath = "BeneficialOwnership/OpenData/Ownership/CustomerData/v1/full/2023/03/15"
# filename = "customerdata_ownership_entity.csv"
# storage_account = "actstaxdatalakedev60"


StatementMeta(, , , Cancelled, )

In [ ]:
## TEST PARAMETERS - MAPPINGS
#Uncomment mapping for desired test

#Ownership - Entity
# mapping = '''{
#     "type":"Open Ownership Entity",
#     "mappings":[
        
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"name"}
#         },
#         {
#             "source":{"sourceColumn":"address"},
#             "sink":{"sinkColumn":"address"}
#         },
#         {
#             "source":{"sourceColumn":"website"},
#             "sink":{"sinkColumn":"website"}
#         },
#         {
#             "source":{"sourceColumn":"ID"},
#             "sink":{"sinkColumn":"EntityID"}
#         }
#     ]
# }'''


# #Ownership - Person
# mapping = '''{
#     "type":"Open Ownership Person",
#     "mappings":[
        
#         {
#             "source":{"sourceColumn":"ID"},
#             "sink":{"sinkColumn":"EntityID"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"Value"}
#         }
        
#     ]
# }'''

# #Contracting_Activity
# mapping = '''{
#     "type":"Open Contracting Activity",
#     "mappings":[
        
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"tender"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"EntityID"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"buyer"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"item"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"lot"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"TimePeriod"}
#         }
#     ]
# }'''



# #Contracting_Contact
# mapping = '''{
#     "type":"Open Contracting Contact",
#     "mappings":[
        
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"EntityID"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"name"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"email"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"address"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"phone"}
#         },
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"website"}
#         }
#     ]
# }''


#Corporate
# mapping = '''{
#     "type":"Open Corporate",
#     "mappings":[
        
#         {
#             "source":{"sourceColumn":"name"},
#             "sink":{"sinkColumn":"name"}
#         },
#         {
#             "source":{"sourceColumn":"address"},
#             "sink":{"sinkColumn":"address"}
#         },
#         {
#             "source":{"sourceColumn":"website"},
#             "sink":{"sinkColumn":"website"}
#         },
#         {
#             "source":{"sourceColumn":"ID"},
#             "sink":{"sinkColumn":"EntityID"}
#         }
#     ]
# }'''



#Sanctions
# mapping = '''{
#     "type":"Open Sanctions",
#     "mappings":[
        
#         {
#             "source":{"sourceColumn":"id"},
#             "sink":{"sinkColumn":"EntityID"}
#         },
#         {
#             "source":{"sourceColumn":"schema"},
#             "sink":{"sinkColumn":"Category"}
#         },
#         {
#             "source":{"sourceColumn":"program"},
#             "sink":{"sinkColumn":"Description"}
#         },
#         {
#             "source":{"sourceColumn":"ID"},
#             "sink":{"sinkColumn":"Evidence"}
#         }
#     ]
# }'''


StatementMeta(, , , Cancelled, )

## Main Function 

In [ ]:
def ProcessCustomerData(mapping, container, storage_account, folderpath, filename, table):

    #Step 1: Read File
    initialDataframe = spark.read.csv( 'abfss://'+container+'@'+ storage_account + '.dfs.core.windows.net/'+ folderpath+'/'+ filename, sep=',',
                         inferSchema=True, header=True)

    #Step 2: Parse Mapping JSON
    mapping_json = json.loads(mapping)
    columnMappingsDict = list(mapping_json['mappings'])
    columnMappings = [(columnMapping['source'], columnMapping['sink']) for columnMapping in columnMappingsDict]

    #Step 3: Rename Columns
    updated_columns = [f.col(mapping[0]).alias(mapping[1]) for mapping in columnMappings]
    newDataframe=initialDataframe.select([c.cast('string') for c in updated_columns])

    #Step 4: Output to Staging in Expected Format
    if sinkdbTable == 'OpenOwnershipEntity' or sinkdbTable == 'OpenCorporate':
        final_df = newDataframe.selectExpr("EntityID", "stack(3, 'name', name, 'website', website, 'address', address) as (AttributeID, Value)").where("EntityID is not null")
        
        if sinkdbTable =='OpenOwnershipEntity':
            final_df.write.mode("overwrite").parquet('abfss://'+'staging'+'@'+ storage_account + '.dfs.core.windows.net/'+ 'BeneficialOwnership/OpenData/Ownership/Entity/OwnershipDF_Processed/CustomerData')
        else:
            final_df.write.mode("overwrite").parquet('abfss://'+'staging'+'@'+ storage_account + '.dfs.core.windows.net/'+ 'BeneficialOwnership/OpenData/Corporate/CorporateDF_Processed/CustomerData')
    
    elif sinkdbTable == 'OpenContractingActivity':
        final_df = newDataframe.selectExpr("TimePeriod", "EntityID", "stack(4, 'tender', tender, 'buyer', buyer, 'item', item, 'lot', lot ) as (AttributeID, Value1)").where("EntityID is not null and TimePeriod is not null")
        final_df.withColumn("Value",f.concat(f.col("AttributeID"),f.lit(':'), f.col('Value1'))).select('TimePeriod','EntityID','AttributeID','Value')
        final_df.write.mode("overwrite").parquet('abfss://'+'staging'+'@'+ storage_account + '.dfs.core.windows.net/'+ 'BeneficialOwnership/OpenData/Contracting/ActivityDF_Processed/CustomerData')
    
    elif sinkdbTable == 'OpenContractingContact':
        final_df = newDataframe.selectExpr("EntityID", "stack(5, 'email', email, 'name', name, 'website', website, 'phone', phone, 'address', address ) as (AttributeID, Value)").where("EntityID is not null ")
        final_df.write.mode("overwrite").parquet('abfss://'+'staging'+'@'+ storage_account + '.dfs.core.windows.net/'+ 'BeneficialOwnership/OpenData/Contracting/ContactDF_Processed/CustomerData')

    elif sinkdbTable == 'OpenOwnershipPerson':
        final_df = newDataframe.withColumn("AttributeID", f.lit('individual partner')).select('EntityID','AttributeID','Value')
        final_df.write.mode("overwrite").parquet('abfss://'+'staging'+'@'+ storage_account + '.dfs.core.windows.net/'+ 'BeneficialOwnership/OpenData/Ownership/Person/OwnershipDF_Processed/CustomerData')

    elif sinkdbTable == 'OpenSanctions':
        final_df = newDataframe.select('EntityID','Category','Description','Evidence')
        final_df.write.mode("overwrite").parquet('abfss://'+'staging'+'@'+ storage_account + '.dfs.core.windows.net/'+ 'BeneficialOwnership/OpenData/Sanctions/SanctionsDF_Processed/CustomerData')


StatementMeta(, , , Cancelled, )

In [ ]:
ProcessCustomerData(mapping, container, storage_account, folderpath, filename, sinkdbTable)

StatementMeta(, , , Cancelled, )